This notebook allows the user to select XML collections and zip them up to send to a service that runs a transform on them and returns a simple CSV made up of six data points. The data included is the Collection name, Dialect name, Record name, Concept name, Content, Xpath location, and the Dialect Definition for the concept. 

The notebook utilizes Bash and Python with the default packages contained in the Mac build of Anaconda with Python 3.6. 

Saxon, Java, and XSLT form the evaluation web service on a NCEAS virtual machine. 

This CSV contains a row for each concept that is found, so some elements may fulfill multiple concepts. A good example of this are the concepts Keyword and Place Keyword. Every Place Keyword is also a Keyword, so the row would repeat with a different Concept name. It also contains a row for each undefined node that contains text, marking these rows with an Unknown in the Concept column. 

This data can be used in a variety of analyses including RAD and QuickE as well as Concept Verticals. It can also be used to teach the system dialect definitions for concepts that are currently unknown by exposing all of the content at undefined nodes. 

## First we need to call all of the libraries we need to perform in our metadata wrangle

In [120]:
import pandas as pd
pd.options.display.width = 180
#import os
from os import walk
import shutil
import ipywidgets as widgets
from ipywidgets import *
import requests
import csv
import io

### Now let's select some metadata. 

If you have prepared metadata\* on your computer that you want to add, it is possible to upload into the repository locally using the [Add Metadata](00AddMetadata.ipynb) Notebook before completing the following cells in this notebook. Otherwise, follow along and use some of the sample metadata the following steps will help you to select.

\* Prepared metadata contains a root element that has a standardized namespace and namespace prefix. Many dialects such as ISO and DIF are consistently written this way, but some dialects such as CSDGM are often written by organizations as only well-formed XML.

Create a list of subdirectories in the collection directory of MILE2 to select metadata for evaluation

In [140]:
Organizations = []
for (dirpath, dirnames, filenames) in walk('../collection/'):
    Organizations.extend(dirnames)
    break  

Create a function to select the organization the metadata comes from

In [141]:
def OrganizationChoices(organization):
    global OrganizationChoice
    global Organization
    Organization=organization
    print("Organization of the collection is", Organization)


Create a dropdown using the Organizations list and the organization selector function. This sets the Organization variable.

In [142]:
interactive(OrganizationChoices, organization=Organizations)

Create a list of collections in the organization directory selected in the dropdown above

In [143]:
Collections = []
for (dirpath, dirnames, filenames) in walk(os.path.join('../collection',Organization)):
    Collections.extend(dirnames)
    break 
Collections

['LTER_2009',
 'LTER_2007',
 'LTER_2006',
 'LTER_2008',
 'LTER_2015',
 'LTER_2012',
 'LTER_2013',
 'LTER_2014',
 'LTER_2005',
 'LTER_2011',
 'LTER_2016',
 'LTER_2010']

Create a function to select the collection the metadata comes from

In [144]:
def CollectionChoices(collection):
    global CollectionChoice
    global Collection
    Collection=collection

Create a dropdown using the Collections list and the organization selector function. This sets the Collection variable.

In [145]:
interactive(CollectionChoices, collection=Collections)

Many organizations support multiple metadata dialects, and share their collections in more than one dialect. This list is created the same way the others are. It adds the different dialects the collection is shared in to a list.

In [221]:
Dialects = []
for (dirpath, dirnames, filenames) in walk(os.path.join('../collection',Organization,Collection)):
    Dialects.extend(dirnames)
    break 
dialectList=Dialects


Create a function to select the dialect you want to send to the evaluator service.

In [222]:
def dialectChoice(dialect):
    global Dialect
    Dialect=dialect
    print("Dialect of the collection is", Dialect)


Create a dropdown using the Dialects list and the dialect selector function. This sets the Dialect variable.

In [223]:
interactive(dialectChoice,dialect=dialectList)

change to the zip directory 

In [240]:
cd ../zip

/Users/scgordon/MILE2/zip


Combine the Organization, Collection, and Dialect variables with the string 'xml' as a relative path and save the string to a variable

In [241]:
MetadataDestination=os.path.join(Organization,Collection,Dialect,'xml')
MetadataDestination

'LTERthroughTime/LTER_2016/EML/xml'

Use the path to create a directory structure in the zip directory

In [242]:
os.makedirs(MetadataDestination, exist_ok=True)

Create a path to the metadata you selected earlier and save the string to a variable, 'MetadataLocation'.

In [243]:
MetadataLocation=os.path.join('../collection/',Organization,Collection,Dialect,'xml')

MetadataLocation

'../collection/LTERthroughTime/LTER_2016/EML/xml'

Copy the metadata to the new directory structure.

In [244]:
src_files = os.listdir(MetadataLocation)
for file_name in src_files:
    full_file_name = os.path.join(MetadataLocation, file_name)
    if (os.path.isfile(full_file_name)):
        shutil.copy(full_file_name, MetadataDestination)

Make a zip file to upload to the evaluator service

In [245]:
shutil.make_archive('../upload/metadata', 'zip', os.getcwd())

'/Users/scgordon/MILE2/upload/metadata.zip'

In [246]:
%cd ../upload 

/Users/scgordon/MILE2/upload


Send metadata to the Evaluator. Get the responses with csv encoding. This step can take up to a minute and doesn't track progress, but a dataframe or an error message will be returned.

Save the dataframe as a csv for further analysis. Copy the csv to a directory, named for the organization that had the metadata in it's holdings. Give it a filename matching the the metadata collection and dialect.

Clear up temporary files and directories, switch to the data directory

In [248]:
# Send metadata package, read the response into a dataframe
url = 'http://metadig.nceas.ucsb.edu/metadata/evaluator'
files = {'zipxml': open('metadata.zip', 'rb')}
r = requests.post(url, files=files, headers={"Accept-Encoding": "gzip"})
r.raise_for_status()
EvaluatedMetadataDF = pd.read_csv(io.StringIO(r.text), quotechar='"')

#build filepaths, directories and file names
Filedirectory=os.path.join('../data/',Organization)
os.makedirs(Filedirectory, exist_ok=True)
Filename='/'+Collection+'_'+Dialect+'_Evaluated.csv.gz'
SimplfiedFilename='/'+Collection+'_'+Dialect+'_SimplifiedEvaluated.csv.gz'
FilePath=Filedirectory+Filename
SimplifiedFilePath=Filedirectory+SimplfiedFilename
EvaluatedMetadataDF.insert(3, 'Collection', Organization+'_'+Collection+'_'+Dialect)
EvaluatedMetadataDF.to_csv(FilePath, mode = 'w', compression='gzip', index=False)

#Change directories, delete upload directory and zip. Delete copied metadata.
%cd ../
shutil.rmtree('upload')
%cd zip
shutil.rmtree(Organization)
%cd ../data

#Create a simplified XPath output
EvaluatedSimplifiedMetadataDF = EvaluatedMetadataDF.copy()
EvaluatedSimplifiedMetadataDF['XPath']=EvaluatedSimplifiedMetadataDF['XPath'].str.replace('/gco:CharacterString', '')
EvaluatedSimplifiedMetadataDF['XPath']=EvaluatedSimplifiedMetadataDF['XPath'].str.replace('/[a-z]+:+?', '/')
EvaluatedSimplifiedMetadataDF['XPath']=EvaluatedSimplifiedMetadataDF['XPath'].str.replace('/[A-Z]+_[A-Za-z]+/?', '/')
EvaluatedSimplifiedMetadataDF['XPath']=EvaluatedSimplifiedMetadataDF['XPath'].str.replace('//', '/')
EvaluatedSimplifiedMetadataDF['XPath']=EvaluatedSimplifiedMetadataDF['XPath'].str.rstrip('//')
EvaluatedSimplifiedMetadataDF.to_csv(SimplifiedFilePath, mode = 'w', compression='gzip', index=False)

/Users/scgordon/MILE2
/Users/scgordon/MILE2/zip
/Users/scgordon/MILE2/data


In [161]:
EvaluatedMetadataDF

Concept                                            Content              Record                     Collection  \
0                                        Abstract  All mangrove trees having a diameter at breast...  00000-metadata.xml  LTERthroughTime_LTER_2006_EML   
1                            Attribute Definition                                  Name of LTER site  00000-metadata.xml  LTERthroughTime_LTER_2006_EML   
2                            Attribute Definition                                    Collection date  00000-metadata.xml  LTERthroughTime_LTER_2006_EML   
3                            Attribute Definition                                     Plot ID Number  00000-metadata.xml  LTERthroughTime_LTER_2006_EML   
4                            Attribute Definition                               Mangrove Tree ID Tag  00000-metadata.xml  LTERthroughTime_LTER_2006_EML   
5                            Attribute Definition                              Mangrove Tree Species  00000-metadata.xml  LTERthroughTime_LTER_2006_EML   
6                            Attribute Definition    Mean Diameter of Mangrove Tree at Breast Height  00000-metadata.xml  LTERthroughTime_LTER_2006_EML   
7                            Attribute Definition                               Mangrove Tree Height  00000-metadata.xml  LTERthroughTime_LTER_2006_EML   
8                                  Attribute List  SITENAME sitename Name of LTER site text Name ...  00000-metadata.xml  LTERthroughTime_LTER_2006_EML   
9                                          Author  Dr. Robert Twilley Wetland Biogeochemistry Ins...  00000-metadata.xml  LTERthroughTime_LTER_2006_EML   
10                                         Author  Dr. Victor Rivera-Monroy Wetland Biogeochemist...  00000-metadata.xml  LTERthroughTime_LTER_2006_EML   
11                                         Author  Mr. Edward Castaneda Wetland Biogeochemistry I...  00000-metadata.xml  LTERthroughTime_LTER_2006_EML   
12                            Author / Originator  Dr. Robert Twilley Wetland Biogeochemistry Ins...  00000-metadata.xml  LTERthroughTime_LTER_2006_EML   
13                            Author / Originator  Dr. Victor Rivera-Monroy Wetland Biogeochemist...  00000-metadata.xml  LTERthroughTime_LTER_2006_EML   
14                            Author / Originator  Mr. Edward Castaneda Wetland Biogeochemistry I...  00000-metadata.xml  LTERthroughTime_LTER_2006_EML   
15              Author / Originator Email Address         http://www.wetlandbiogeochemistry.lsu.edu/  00000-metadata.xml  LTERthroughTime_LTER_2006_EML   
16              Author / Originator Email Address                   http://www.oceanography.lsu.edu/  00000-metadata.xml  LTERthroughTime_LTER_2006_EML   
17     Author / Originator World Wide Web Address                                   rtwilley@lsu.edu  00000-metadata.xml  LTERthroughTime_LTER_2006_EML   
18     Author / Originator World Wide Web Address                                   vhrivera@lsu.edu  00000-metadata.xml  LTERthroughTime_LTER_2006_EML   
19     Author / Originator World Wide Web Address                                    ecasta1@lsu.edu  00000-metadata.xml  LTERthroughTime_LTER_2006_EML   
20                                   Bounding Box  -81.07794623 -80.52455665 25.40976421 25.23269749  00000-metadata.xml  LTERthroughTime_LTER_2006_EML   
21                                   Bounding Box  -81.07794623 -80.48978207 25.76145171 24.91293492  00000-metadata.xml  LTERthroughTime_LTER_2006_EML   
22                               Contributor Name  Victor Rivera-Monroy Wetland Biogeochemistry I...  00000-metadata.xml  LTERthroughTime_LTER_2006_EML   
23                               Contributor Name  Robert Twilley Wetland Biogeochemistry Institu...  00000-metadata.xml  LTERthroughTime_LTER_2006_EML   
24                               Contributor Name  Edward Castaneda Wetland Biogeochemistry Insti...  00000-metadata.xml  LTERthroughTime_LTER_2006_EML   
25          

Now that we have our metadata raw data prepared and stored, we can prepare the collection's data for recommendation analytics and cross collection analytics.

Create a table with each record as a row of concept occurance counts. Each concept that occurs in the collection is a row.

In [100]:
FiledirectoryRAD=os.path.join('../data/',Organization)
FilenameRAD='/'+Collection+'_'+Dialect+'_RAD.csv'
FilePathRAD=FiledirectoryRAD+FilenameRAD
group_name = EvaluatedSimplifiedMetadataDF.groupby(['Record', 'Concept'], as_index=False)
occuranceMatrix=group_name.size().unstack()
occuranceMatrix=occuranceMatrix.fillna(0)
pd.options.display.float_format = '{:,.0f}'.format
occuranceMatrix.to_csv(FilePathRAD, mode = 'w', index=False)
occuranceMatrix

Concept             Abstract  Attribute Constraints  Attribute Definition  Attribute List  Author  Author / Originator  Author / Originator Email Address  \
Record                                                                                                                                                      
00000-metadata.xml         1                      0                    83               2       2                    2                                  2   
00001-metadata.xml         1                      0                    11               1       1                    1                                  0   
00002-metadata.xml         1                      0                    42               5       2                    2                                  0   
00003-metadata.xml         1                      0                     7               1       1                    1                                  0   
00004-metadata.xml         1                      0                     7               1       4                    4                                  3   
00005-metadata.xml         1                      0                    10               1       1                    1                                  0   
00006-metadata.xml         1                      0                     8               1       2                    2                                  0   
00007-metadata.xml         1                      0                     0               0       2                    2                                  0   
00008-metadata.xml         1                      0                    12               1       2                    2                                  0   
00009-metadata.xml         1                      0                     5               1       1                    1                                  0   
00010-metadata.xml         1                      0                    13               1       5                    5                                  0   
00011-metadata.xml         1                      0                     8               1       2                    2                                  0   
00012-metadata.xml         1                      0                     5               1       2                    2                                  0   
00013-metadata.xml         1                      0                     8               1       1                    1                                  0   
00014-metadata.xml         1                      0                     8               1       5                    5                                  3   
00015-metadata.xml         1                      0                    56               1       1                    1                                  0   
00016-metadata.xml         1                      0                    26               2       1                    1                                  1   
00017-metadata.xml         1                      0                    12               1       1                    1                                  0   
00018-metadata.xml         1                      0                    11               1       1                    1                                  0   
00019-metadata.xml         1                      0                    58               1       2                    2                                  0   
00020-metadata.xml         1                      0                    34               1       1                    1                                  0   
00021-metadata.xml         1                      0                    12               1       1                    1                                  0   
00022-metadata.xml         1                      0                    14               1       1                    1                                  0   
00023-metadata.xml         1                      0                     7  

Get RecTags csv into dataframe. use widget to select recommendation return that data series and split each string into the concept name and the recommendation profile. Add column of the concept occurance percentage. Add zeros for concepts that dont have a CO%. Use dialect contains to determine if the cell should be red or yellow. color cell green if CO% is 100%
count concepts for each recommendation profile and plot a line between them. Subtract the red cells from each profile and plot a line along the resultant totals. Label the x axis with profile names. y axis is concept count. make one solid one dashed line


count if greater than zero in certain columns

In [77]:
FiledirectoryQuickE=os.path.join('../data/',Organization)
FilenameQuickE='/'+Collection+'_'+Dialect+'_QuickE.csv'
FilePathQuickE=FiledirectoryQuickE+FilenameQuickE
group_name = EvaluatedSimplifiedMetadataDF.groupby(['XPath', 'Record'], as_index=False)
QuickEdf=group_name.size().unstack().reset_index()
QuickEdf=QuickEdf.fillna(0)
pd.options.display.float_format = '{:,.0f}'.format
QuickEdf.to_csv(FilePathQuickE, mode = 'w', index=False)
QuickEdf

Record                                              XPath  dataset_3470.xml  dataset_3484.xml  dataset_3485.xml  dataset_3486.xml  dataset_3508.xml  dataset_3510.xml  \
0                                    /@xsi:schemaLocation                 1                 1                 1                 1                 1                 1   
1                      /acquisitionInformation/instrument                 1                 1                 0                 0                 1                 1   
2          /acquisitionInformation/instrument/description                 1                 1                 0                 0                 1                 1   
3       /acquisitionInformation/instrument/identifier/...                 2                 2                 0                 0                 2                 2   
4       /acquisitionInformation/instrument/identifier/...                 1                 1                 0                 0                 1                 1   
5       /acquisitionInformation/instrument/identifier/...                 1                 1                 0                 0                 1                 1   
6       /acquisitionInformation/instrument/identifier/...                 1                 1                 0                 0                 1                 1   
7       /acquisitionInformation/instrument/identifier/...                 1                 1                 0                 0                 1                 1   
8                 /acquisitionInformation/instrument/type                 2                 2                 0                 0                 2                 2   
9           /acquisitionInformation/operation/description                 2                 2                 1                 1                 1                 1   
10      /acquisitionInformation/operation/identifier/code                 2                 2                 1                 1                 1                 1   
11      /acquisitionInformation/operation/identifier/c...                 2                 2                 1                 1                 1                 1   
12      /acquisitionInformation/operation/identifier/c...                 1                 1                 1                 1                 1                 1   
13      /acquisitionInformation/operation/identifier/c...                 2                 2                 1                 1                 1                 1   
14      /acquisitionInformation/operation/identifier/c...                 1                 1                 1                 1                 1                 1   
15      /acquisitionInformation/operation/parentOperat...                 1                 1                 1                 1                 1                 1   
16      /acquisitionInformation/operation/plan/citatio...                 2                 2                 1                 1                 1                 1   
17      /acquisitionInformation/operation/plan/citatio...                 1                 1                 1                 1                 1                 1   
18      /acquisitionInformation/operation/plan/citatio...                 2                 2                 2                 2                 2                 2   
19      /acquisitionInformation/operation/plan/citatio...                 2                 2                 2                 2                 2                 2   
20      /acquisitionInformation/operation/plan/citatio...                 1                 1                 1                 1                 1                 1   
21      /acquisitionInformation/operation/plan/citatio...                 4                 4                 4                 4                 4                 4   
22      /acquisitionInformation/operation/plan/citatio...                 1                 1                 1  

In [79]:
FiledirectoryOccurance=os.path.join('../data/',Organization)
FilenameOccurance='/'+Collection+'_'+Dialect+'_Occurance.csv'
FilePathOccurance=FiledirectoryOccurance+FilenameOccurance
occuranceSum=occuranceMatrix.sum()
occuranceCount=occuranceMatrix[occuranceMatrix!=0].count()
CollectionName=FilenameOccurance.partition("/")[2].partition("_Occurance.csv")[0]
result = pd.concat([occuranceSum, occuranceCount], axis=1).reset_index()
result.insert(1, 'Collection', CollectionName)
result.insert(4, 'CollectionOccurance%', CollectionName)
#result.insert(4, 'AverageOccurancePerRecord', CollectionName)
result.columns = ['Concept', 'Collection', 'ConceptCount', 'RecordCount', 'AverageOccurancePerRecord', 'CollectionOccurance%' ]
NumberOfRecords = result.at[0, 'RecordCount']
result['CollectionOccurance%'] = result['RecordCount']/NumberOfRecords
result['CollectionOccurance%'] = pd.Series(["{0:.2f}%".format(val * 100) for val in result['CollectionOccurance%']], index = result.index)
result.at[0, 'ConceptCount'] = NumberOfRecords
result.at[0, 'Concept'] = 'Number of Records'
result['AverageOccurancePerRecord'] = result['ConceptCount']/NumberOfRecords
result['AverageOccurancePerRecord'] = result['AverageOccurancePerRecord'].astype(float)
result[["ConceptCount","RecordCount"]] = result[["ConceptCount","RecordCount"]].astype(int)
result['AverageOccurancePerRecord'] = pd.Series(["{0:.2f}".format(val) for val in result['AverageOccurancePerRecord']], index = result.index)
result.to_csv(FilePathOccurance, mode = 'w', index=False)
result

KeyError: 'RecordCount'

### Select the notebook that prepares the data for different types of analysis

* [Create RAD Data](02RADdf.ipynb)
* [Cross Collection Comparisons](03CrossCollectionComparisons.ipynb)
* [Concept Content Consistency](04ConceptVerticals.ipynb)
* [Exploring Unknown Concepts](05ExploringUnknownConcepts.ipynb)